In [ ]:
def extract_table_names(column_array):
    table_names = set()
    
    for col in column_array:
        if not isinstance(col, str):
            continue
            
        # Split by spaces, newlines and commas to handle multi-line entries
        parts = col.replace('\n', ' ').replace(',', ' ').split()
        
        for part in parts:
            # Look for table.column pattern
            if '.' in part:
                table = part.split('.')[0]
                # Some cleaning to handle edge cases
                table = table.strip("' ")
                if table:
                    table_names.add(table)
    
    return sorted(list(table_names))

def generate_bulk_insert_statement(insert_header,table_names, target_table, source_sis_type="ESP", source_sis_version="sample_tenant_uid_4"):

    # Create each row value
    value_rows = []
    for source_table in table_names:
        value_row = f"('{target_table}', '{source_table}', '{source_sis_type}', '{source_sis_version}')"
        value_rows.append(value_row)
    
    # Join all values with commas and add semicolon at the end
    values_part = ",\n".join(value_rows) + ";"
    
    # Combine header and values
    return insert_header + "\n" + values_part


def execute_sql(sql_statement):
    """
    Execute a SQL statement in Snowflake and return the results.
    Args:
        sql_statement (str): The SQL statement to execute
    Returns:
        list: Results from the query execution
    """
    try:
        # Establish connection to Snowflake
        conn = sf_connection()
        cur = conn.cursor()
        
        # Execute the SQL statement
        cur.execute(sql_statement)
        
        # For SELECT statements, fetch results
        if sql_statement.strip().upper().startswith("SELECT"):
            results = cur.fetchall()
            return results
        else:
            # For INSERT/UPDATE/DELETE statements, return affected row count
            return f"Statement executed successfully. Rows affected: {cur.rowcount}"
            
    except Exception as e:
        return f"Error executing SQL: {str(e)}"
    finally:
        # Close cursor and connection
        if cur:
            cur.close()
        if conn:
            conn.close()

In [ ]:
insert_header = f"""INSERT INTO PSESSENTIALS.SG_ESPSTAGING2.TABLES_IDENTIFIED_DATA_MAPPING 
(TARGET_TABLE, IDENTIFIED_SOURCE_TABLE, SOURCE_SIS_TYPE, SOURCE_SIS_VERSION)
VALUES """

file_path ='/Users/gaurav.adlakha/wks/ess/esp_data_mod.xlsx' 
col_name = 'Eschool'


In [ ]:
import pandas as pd

In [ ]:
xls = pd.ExcelFile(file_path)
xls.sheet_names

['Students',
 'Re-enrollment',
 'Immunization',
 'Historical_Grades',
 'Test Scores',
 'Staff_Teacher',
 'Courses',
 'Section',
 'Section Enrollment',
 'Rooms']

In [ ]:
def process_excel_for_sql(file_path, col_name, source_sis_type="ESP", source_sis_version="sample_tenant_uid_4"):
    "Process Excel file to generate and execute SQL statements for table mapping"
    xls = pd.ExcelFile(file_path)
    insert_header = """INSERT INTO PSESSENTIALS.SG_ESPSTAGING2.TABLES_IDENTIFIED_DATA_MAPPING 
(TARGET_TABLE, IDENTIFIED_SOURCE_TABLE, SOURCE_SIS_TYPE, SOURCE_SIS_VERSION)
VALUES """
    results = {}
    
    for sis_table in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sis_table)
        table_names = extract_table_names(df[col_name].values)
        table_names = [name.replace('REG_ENTRY_WITH', 'REG_ENTRY_WITHDRAWL') for name in table_names]
        print(table_names)
        sql = generate_bulk_insert_statement(insert_header, table_names, sis_table, source_sis_type, source_sis_version)
        print(sql)
        # results[sis_table] = execute_sql(sql) if table_names else "No tables found"
    
    return results

In [ ]:
test_result = process_excel_for_sql('esp_data_mod.xlsx', 'Eschool')

['REG', 'REG2', 'REGTB_RELATION', 'REG_ACADEMIC', 'REG_CONTACT', 'REG_CONTACT_PHONE', 'REG_ENTRY_WITHDRAWL', 'REG_NEXT_YEAR', 'REG_PERSONAL', 'REG_PERSONAL3', 'REG_STU_CONTACT']
INSERT INTO PSESSENTIALS.SG_ESPSTAGING2.TABLES_IDENTIFIED_DATA_MAPPING 
(TARGET_TABLE, IDENTIFIED_SOURCE_TABLE, SOURCE_SIS_TYPE, SOURCE_SIS_VERSION)
VALUES 
('Students', 'REG', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG2', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REGTB_RELATION', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_ACADEMIC', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_CONTACT', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_CONTACT_PHONE', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_ENTRY_WITHDRAWL', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_NEXT_YEAR', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_PERSONAL', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_PERSONAL3', 'ESP', 'sample_tenant_uid_4'),
('Students', 'REG_STU_CONTACT', 'ESP', 'sample_tenant_uid_4');
['RE